In [ ]:
!pip install fairseq
!pip install pandas torchaudio sentencepiece
!pip install datasets

In [ ]:
!sudo apt-get -y install libsox-dev
!sudo apt-get install sox






In [ ]:
!git clone https://github.com/facebookresearch/fairseq.git

In [ ]:
%cd /content/fairseq

In [ ]:
!pip install --editable ./

In [ ]:
!python setup.py build develop

In [ ]:
%cd /content/fairseq/examples/speech_to_text

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!ls

In [ ]:
# !python prep_librispeech_data.py \
#   --output-root /content/commonvoice --vocab-type unigram --vocab-size 2000

In [ ]:
!tar -xvf /content/drive/MyDrive/'MNLP HW2'/cv-corpus-15.0-2023-09-08-quy.tar.gz

In [ ]:
!tar -xvf /content/drive/MyDrive/'MNLP HW2'/cv-corpus-15.0-2023-09-08-gn.tar.gz

In [ ]:
!ls /content/drive/MyDrive/'MNLP HW2'/

In [ ]:
# !pip install pysox

In [ ]:
!python prep_commonvoice_data.py \
  --output-root /content/commonvoice/ --vocab-type unigram --vocab-size 2000 --input-dir /content/fairseq/examples/speech_to_text/cv-corpus-15.0-2023-09-08/gn

In [ ]:
!python prep_commonvoice_data.py \
  --output-root /content/commonvoice/quy/ --vocab-type unigram --vocab-size 19 --input-dir /content/fairseq/examples/speech_to_text/cv-corpus-15.0-2023-09-08/quy

In [ ]:
!fairseq-train /content/commonvoice --save-dir /content/commonvoice/save \
  --config-yaml config.yaml --train-subset train --valid-subset validation \
  --num-workers 4 --max-tokens 40000 --max-update 300000 \
  --task speech_to_text --criterion label_smoothed_cross_entropy --label-smoothing 0.1 --report-accuracy \
  --arch s2t_transformer_s --share-decoder-input-output-embed \
  --optimizer adam --lr 2e-3 --lr-scheduler inverse_sqrt --warmup-updates 10000 \
  --clip-norm 10.0 --seed 1 --update-freq 8 --max-epoch 10 --log-format tqdm

In [ ]:
LS_ROOT = '/content/commonvoice/gn_final'
SAVE_DIR = '/content/commonvoice/gn_final/save'
CHECKPOINT_FILENAME = 'checkpoint_best.pt'
SUBSETS = ['test','validation']

# Loop over subsets
for SUBSET in SUBSETS:
    !fairseq-generate {LS_ROOT} --config-yaml config.yaml --gen-subset {SUBSET} \
    --task speech_to_text --path {SAVE_DIR}/{CHECKPOINT_FILENAME} \
    --max-tokens 50000 --beam 5 --scoring wer